In [1]:
import base64

import solana.rpc.api


%load_ext autoreload
%autoreload 2

In [29]:
# DOES NOT RETURN THE SAME OBJECT. Some transactions would not have data hash e.g.:
# without hash:
# Block number = 242197066.
# Timestamp = 1705424747.
# Transaction hash = ufSrdp8F61GAZ7TvM8g2hCrDVLM3NFjBjvMmdcnNMJGByg1v5mBbEyNCrb8ZWQoJsKSEEbYp2G9fePsv5YHbarQ.
# with hash:
# Block number = 242198179.
# Timestamp = 1705425225.
# Transaction hash = 4RJ4r1irMjuPdysHBNFQBnWCTE7AuZe4fMBJjivStfACkV1m2r1USW9ZXK7Xpdg9fW5JcEqRr8PggNQAgZPTHcWr.


solana_client = solana.rpc.api.Client("https://docs-demo.solana-mainnet.quiknode.pro/") # RPC url - now it's just some demo i found on internet


res = solana_client.get_signatures_for_address(
    solana.rpc.api.Pubkey.from_string('4MangoMjqJ2firMokCjjGgoK8d4MXcrgL7XJaL3w6fVg'), # Address 
    limit = 100,
    # before =  # pass some signature here to fetch txs before that one (when fetching more distant history)
)

transaction = solana_client.get_transaction(
    res.value[0].signature, 
    'jsonParsed', 
    max_supported_transaction_version=0
)

In [26]:
# Explore a single transaction.
transaction

GetTransactionResp(
    Some(
        EncodedConfirmedTransactionWithStatusMeta {
            slot: 242198179,
            transaction: EncodedTransactionWithStatusMeta(
                EncodedTransactionWithStatusMeta {
                    transaction: Json(
                        UiTransaction {
                            signatures: [
                                "4RJ4r1irMjuPdysHBNFQBnWCTE7AuZe4fMBJjivStfACkV1m2r1USW9ZXK7Xpdg9fW5JcEqRr8PggNQAgZPTHcWr",
                            ],
                            message: Parsed(
                                UiParsedMessage {
                                    account_keys: [
                                        ParsedAccount {
                                            pubkey: "Ap5pxfhTsW8bW4SvbezbrGdaSWRDmNSMycgCu11ba4i",
                                            writable: true,
                                            signer: true,
                                            source: Some(
                         

In [27]:
# TODO: Should we also care about 'Program logged: '?
DATA_SUBSTRING = 'Program data: '


# Extract relevant information.
print('***GENERAL***')
print(f'Block number = {transaction.value.slot}.')
print(f'Timestamp = {transaction.value.block_time}.')
print(f'Transaction hash = {transaction.value.transaction.transaction.signatures[0]}.')

print('\n***INSTRUCTIONS***')
for i, instruction in enumerate(transaction.value.transaction.transaction.message.instructions):
    # TODO: Would it be useful to filter relevant data based in these addresses?
    print(f'Instruction = {i}, address = {instruction.program_id}.')
    # TODO: This might be useful, but needs to be decoded first.
    print(f'Instruction = {i}, data = {instruction.data}.')

print('\n***META***')
# TODO: The lengths (and thus also likely the order) of the balances correspond to the length of 
# `transaction.value.transaction.transaction.message.account_keys`, but are the balances relevant?
print(f'Pre-balances = {transaction.value.transaction.meta.pre_balances}.')
print(f'Post-balances = {transaction.value.transaction.meta.post_balances}.')
data_logs = [x for x in transaction.value.transaction.meta.log_messages if DATA_SUBSTRING in x]
assert len(data_logs) == 1
data_hash = data_logs[0].replace(DATA_SUBSTRING, '')
print(f'Data hash = {data_hash}.')

***GENERAL***
Block number = 242198179.
Timestamp = 1705425225.
Transaction hash = 4RJ4r1irMjuPdysHBNFQBnWCTE7AuZe4fMBJjivStfACkV1m2r1USW9ZXK7Xpdg9fW5JcEqRr8PggNQAgZPTHcWr.

***INSTRUCTIONS***
Instruction = 0, address = ComputeBudget111111111111111111111111111111.
Instruction = 0, data = 3DdGGhkhJbjm.
Instruction = 1, address = 4MangoMjqJ2firMokCjjGgoK8d4MXcrgL7XJaL3w6fVg.
Instruction = 1, data = 5hLyoWnwMRF.

***META***
Pre-balances = [6514414954, 1, 1141440, 24490240, 861982080, 861982080, 19989121, 23942400].
Post-balances = [6514409953, 1, 1141440, 24490240, 861982080, 861982080, 19989121, 23942400].
Data hash = dZKDSrOa27JbF8fIam5zn68XUYGDY+lPkIvzcATObT+88Ze90vUfHAMAz0K8nI2HmB0AAAAAAAAAAM9CvJyNh5gdAAAAAAAAAADAqMbYX1gICgAAAAAAAAAAoqZvDgAAAAAiqkLuURgCAAAAAAAAAAAAAACpxthfWAgKAAAAAAAAAAC4oLFvxbC1G1FZAAAAAAAASPdg8Okq6H/PggAAAAAAAOThEAAAAAAAgWob3tr//////////////w==.


In [28]:
log_messages = transaction.value.transaction.meta.log_messages

program_data = next((i for i in log_messages if i.startswith('Program data: ')))
encoded_data = program_data.split(' ')[-1]

print(f'{DATA_SUBSTRING} {encoded_data}')

decoded_bytes = base64.b64decode(encoded_data).hex()
print(f'Decoded program data: {decoded_bytes}')

Program data:  dZKDSrOa27JbF8fIam5zn68XUYGDY+lPkIvzcATObT+88Ze90vUfHAMAz0K8nI2HmB0AAAAAAAAAAM9CvJyNh5gdAAAAAAAAAADAqMbYX1gICgAAAAAAAAAAoqZvDgAAAAAiqkLuURgCAAAAAAAAAAAAAACpxthfWAgKAAAAAAAAAAC4oLFvxbC1G1FZAAAAAAAASPdg8Okq6H/PggAAAAAAAOThEAAAAAAAgWob3tr//////////////w==
Decoded program data: 7592834ab39adbb25b17c7c86a6e739faf1751818363e94f908bf37004ce6d3fbcf197bdd2f51f1c0300cf42bc9c8d87981d0000000000000000cf42bc9c8d87981d0000000000000000c0a8c6d85f58080a0000000000000000a2a66f0e0000000022aa42ee5118020000000000000000000000a9c6d85f58080a0000000000000000b8a0b16fc5b0b51b515900000000000048f760f0e92ae87fcf82000000000000e4e1100000000000816a1bdedaffffffffffffffffffffff
